In [1]:
import networkx as nx
from pathlib import Path

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from flinspect.parse_tree import ParseTree
from flinspect.parse_forest import ParseForest

In [4]:
build_dirs = [
    "/glade/work/altuntas/turbo-stack/bin/flang_ptree/TIM/",
    "/glade/work/altuntas/turbo-stack/bin/flang_ptree/MOM6-infra",
    "/glade/work/altuntas/turbo-stack/bin/flang_ptree/MOM6"
]

parse_tree_paths = [f for dir in build_dirs for f in Path(dir).glob("*_ptree")]

In [5]:
pf = ParseForest(parse_tree_paths)

In [6]:
nr = pf.registry

# Module Dependency Analysis

In [7]:
G = pf.get_module_dependency_graph()

Skipped 8 modules with unknown parse tree paths: ['netcdf', 'iso_c_binding', 'iso_fortran_env', 'mpi', 'netcdf_nf_data', 'netcdf_nf_interfaces', 'netcdf4_nf_interfaces', 'omp_lib']


In [8]:
# Determine source paths via the path_names files in build dirs
# This will be used to color nodes and edges based on where modules are located
path_names = {}
for build_dir in build_dirs:
    path_names_file = Path(build_dir)/"path_names"
    with open(path_names_file) as f:
        for line in f:
            src_file_path = Path(line.strip())
            path_names[src_file_path.stem.lower()] = src_file_path.as_posix()

In [9]:
# Find all nodes where /TIM/ is in the source path
tim_nodes = [node for node in G.nodes() if "/TIM/" in path_names.get(G.nodes[node].get('source_name','').lower(),'')]

In [10]:
# Find all tim_nodes that have no incoming edges (i.e., no dependencies)
tim_root_nodes = [node for node in tim_nodes if G.in_degree(node) == 0]

for tim_root_node in tim_root_nodes:
    print(tim_root_node, path_names.get(G.nodes[tim_root_node].get('source_name','').lower(),''))

fms /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/libFMS.F90


In [11]:
# Find all TIM nodes that is not needed by any other TIM nodes
tim_leaf_nodes = [node for node in tim_nodes if G.out_degree(node) == 0]

for tim_leaf_node in tim_leaf_nodes:
    print(tim_leaf_node, path_names.get(G.nodes[tim_leaf_node].get('source_name','').lower(),''))

platform_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/platform/platform.F90
mersennetwister_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/random_numbers/mersennetwister.F90


In [12]:
# Find all MOM6 nodes:
mom6_nodes = [node for node in G.nodes() if "/MOM6/" in path_names.get(G.nodes[node].get('source_name','').lower(),'')]

# Find all TIM nodes that is not needed by any other MOM6 nodes recursively (i.e. transitive closure)
mom6_deps = set()
for mom6_node in mom6_nodes:
    mom6_deps.update(nx.descendants(G, mom6_node))

# Find all TIM nodes that are not needed by any other MOM6 nodes
tim_leaf_nodes = [node for node in tim_nodes if all(pred not in mom6_deps for pred in G.predecessors(node))]

for tim_leaf_node in tim_leaf_nodes:
    print(tim_leaf_node, path_names.get(G.nodes[tim_leaf_node].get('source_name','').lower(),''))

fms /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/libFMS.F90
block_control_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/block_control/block_control.F90


# Call Graph Analysis

In [13]:
path_names = {}
for build_dir in build_dirs:
    path_names_file = Path(build_dir)/"path_names"
    with open(path_names_file) as f:
        for line in f:
            src_file_path = Path(line.strip())
            path_names[src_file_path.stem.lower()] = src_file_path.as_posix()

In [14]:
gsc = pf.get_call_graph()

Total unfound calls across all parse trees: 174
Total unfound function calls across all parse trees: 70403


In [15]:
# Find all TIM subroutines and MOM6 subroutines
all_tim_routines = []
all_mom6_routines = []
for node in gsc.nodes():
    try:
        module_path = node.program_unit.parse_tree_path
        source_path = path_names.get(module_path.stem.lower(),'')
    except AttributeError:
        continue
    if "/MOM6/" in source_path:
        all_mom6_routines.append(node)
    elif "/TIM/" in source_path:
        all_tim_routines.append(node)

In [16]:
len(all_mom6_routines), len(all_tim_routines)

(3525, 2086)

In [17]:
def get_tim_subroutines_needed_by_mom6(gsc, all_mom6_routines):
    tim_subroutines_needed_by_mom6 = set()
    for mom6_routine in all_mom6_routines:
        descendants = nx.descendants(gsc, mom6_routine)
        for desc in descendants:
            try:
                module_path = desc.program_unit.parse_tree_path
                source_path = path_names.get(module_path.stem.lower(),'')
            except AttributeError:
                continue
            if "/TIM/" in source_path:
                tim_subroutines_needed_by_mom6.add(desc)
    return tim_subroutines_needed_by_mom6

In [18]:
tim_subroutines_needed_by_mom6 = get_tim_subroutines_needed_by_mom6(gsc, all_mom6_routines)

In [19]:
len(tim_subroutines_needed_by_mom6)

1667

In [20]:
tim_subroutines_NOT_needed_by_mom6 = [r for r in all_tim_routines if r not in tim_subroutines_needed_by_mom6]
len(tim_subroutines_NOT_needed_by_mom6)

648

In [21]:
# find all TIM subroutines that are not needed by MOM6 and are not called by any other TIM subroutines
tim_subroutines_NOT_needed_by_mom6_and_not_called_by_tim = []
for routine in tim_subroutines_NOT_needed_by_mom6:
    callers = list(gsc.predecessors(routine))
    called_by_tim = False
    for caller in callers:
        try:
            module_path = caller.program_unit.parse_tree_path
            source_path = path_names.get(module_path.stem.lower(),'')
        except AttributeError:
            continue
        if "/TIM/" in source_path:
            called_by_tim = True
            break
    if not called_by_tim:
        tim_subroutines_NOT_needed_by_mom6_and_not_called_by_tim.append(routine)

In [22]:
len(tim_subroutines_NOT_needed_by_mom6_and_not_called_by_tim)

342

In [35]:
for routine in tim_subroutines_NOT_needed_by_mom6_and_not_called_by_tim:
    #if 'fms_io' in routine.program_unit.name:
    if 'mem' in routine.name:
        print(routine.name)

mpp_memuse_begin
mpp_memuse_end


In [29]:
tim_subroutines_NOT_needed_by_mom6_and_not_called_by_tim[10:]

[Subroutine('set_orbital_parameters'),
 Subroutine('get_orbital_parameters'),
 Subroutine('set_ref_date_of_ae'),
 Subroutine('get_ref_date_of_ae'),
 Subroutine('annual_mean_solar_1d'),
 Subroutine('annual_mean_solar_2level'),
 Subroutine('astronomy_end'),
 Subroutine('finalize_randomnumbersequence'),
 Subroutine('reset_bounds'),
 Subroutine('update_bounds'),
 Subroutine('reset_bounds_from_array_4d'),
 Subroutine('reset_bounds_from_array_5d'),
 Subroutine('fieldbuff_copy_missvals_3d_r4'),
 Subroutine('fieldbuff_copy_missvals_3d_r8'),
 Subroutine('diag_field_add_cell_measures'),
 Subroutine('openmp_thread_trap'),
 Subroutine('append_to_list'),
 Subroutine('destroy_list'),
 Subroutine('nullify_filename_appendix'),
 Subroutine('mpp_set_super_grid_indices'),
 Subroutine('mpp_copy_domain2d'),
 Subroutine('mpp_clear_group_update'),
 Subroutine('mpp_define_layout2d'),
 Subroutine('mpp_define_mosaic'),
 Subroutine('mpp_check_field_3d'),
 Subroutine('mosaic_init'),
 Subroutine('get_mosaic_contac

In [30]:
#group unnecessary subroutines by modules they belong to:
from collections import defaultdict
unnecessary_subroutines_by_module = defaultdict(list)
for subroutine in tim_subroutines_NOT_needed_by_mom6:
    module_path = subroutine.program_unit.parse_tree_path
    source_path = path_names.get(module_path.stem.lower(),'')
    unnecessary_subroutines_by_module[source_path].append(subroutine)

In [31]:
# sort modules by ratio of unnecessary subroutines to total subroutines
module_unnecessary_ratio = []
for module_path, subroutines in unnecessary_subroutines_by_module.items():
    total_subroutines = [r for r in all_tim_routines if path_names.get(r.program_unit.parse_tree_path.stem.lower(),'') == module_path]
    ratio = len(subroutines) / len(total_subroutines)
    module_unnecessary_ratio.append((module_path, ratio, len(subroutines), len(total_subroutines)))

module_unnecessary_ratio.sort(key=lambda x: x[1], reverse=True)
for module in module_unnecessary_ratio[:30]:
    print(f"Module: {module[0]}, Unnecessary Ratio: {module[1]:.2f}, Unnecessary Count: {module[2]}, Total Count: {module[3]}")

Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/diag_manager/fms_diag_bbox.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 10, Total Count: 10
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/diag_manager/fms_diag_outfield.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 29, Total Count: 29
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/affinity/fms_affinity.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 5, Total Count: 5
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/block_control/block_control.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 2, Total Count: 2
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/constants/fmsconstants.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 1, Total Count: 1
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/diag_manager/fms_diag_time_reduction.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 13, Total Count: 13
Module: /g

In [27]:
# sort modules by ratio of unnecessary subroutines to total subroutines
module_unnecessary_ratio = []
for module_path, subroutines in unnecessary_subroutines_by_module.items():
    total_subroutines = [r for r in all_tim_routines if path_names.get(r.program_unit.parse_tree_path.stem.lower(),'') == module_path]
    ratio = len(subroutines) / len(total_subroutines)
    module_unnecessary_ratio.append((module_path, ratio, len(subroutines), len(total_subroutines)))

module_unnecessary_ratio.sort(key=lambda x: x[1], reverse=True)
for module in module_unnecessary_ratio[:30]:
    print(f"Module: {module[0]}, Unnecessary Ratio: {module[1]:.2f}, Unnecessary Count: {module[2]}, Total Count: {module[3]}")

Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/diag_manager/fms_diag_bbox.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 10, Total Count: 10
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/diag_manager/fms_diag_outfield.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 29, Total Count: 29
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/affinity/fms_affinity.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 5, Total Count: 5
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/block_control/block_control.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 2, Total Count: 2
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/constants/fmsconstants.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 1, Total Count: 1
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/diag_manager/fms_diag_time_reduction.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 13, Total Count: 13
Module: /g

In [28]:
# sort modules by ratio of unnecessary subroutines to total subroutines
module_unnecessary_ratio = []
for module_path, subroutines in unnecessary_subroutines_by_module.items():
    total_subroutines = [r for r in all_tim_routines if path_names.get(r.program_unit.parse_tree_path.stem.lower(),'') == module_path]
    ratio = len(subroutines) / len(total_subroutines)
    module_unnecessary_ratio.append((module_path, ratio, len(subroutines), len(total_subroutines)))

module_unnecessary_ratio.sort(key=lambda x: x[2], reverse=True)
for module in module_unnecessary_ratio[:30]:
    print(f"Module: {module[0]}, Unnecessary Ratio: {module[1]:.2f}, Unnecessary Count: {module[2]}, Total Count: {module[3]}")

Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/fms/fms_io.F90, Unnecessary Ratio: 0.77, Unnecessary Count: 122, Total Count: 158
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/mpp/mpp_domains.F90, Unnecessary Ratio: 0.27, Unnecessary Count: 121, Total Count: 447
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/mpp/mpp_io.F90, Unnecessary Ratio: 0.58, Unnecessary Count: 84, Total Count: 144
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/fms2_io/netcdf_io.F90, Unnecessary Ratio: 0.26, Unnecessary Count: 32, Total Count: 124
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/mpp/mpp.F90, Unnecessary Ratio: 0.08, Unnecessary Count: 31, Total Count: 377
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/time_manager/time_manager.F90, Unnecessary Ratio: 0.35, Unnecessary Count: 30, Total Count: 85
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/TIM/diag_m